In [1]:
import numpy as np
np.random.seed(42)
import sys
import cProfile
import humanize
sys.path.insert(0, '../..')
%reload_ext memory_profiler
%reload_ext autoreload
%autoreload 1
%aimport allel.model
%aimport allel.stats
%aimport allel.opt.stats

In [2]:
gn = np.array([[0, 0, 1, 0, 2, 0, 1, 1, 0],
               [1, 0, 0, 1, 1, 2, 0, 0, 2]], dtype='i1')

In [3]:
np.corrcoef(gn)

array([[ 1.        , -0.39000675],
       [-0.39000675,  1.        ]])

In [4]:
gn_sq = gn ** 2

In [5]:
allel.opt.stats.gn_corrcoef_int8(gn[0], gn[1], gn_sq[0], gn_sq[1])

-0.39000681042671204

In [6]:
gnb = np.random.randint(0, 3, size=(2, 10000)).astype('i1')

In [7]:
np.corrcoef(gnb)

array([[ 1.      , -0.002613],
       [-0.002613,  1.      ]])

In [8]:
gnb_sq = gnb ** 2

In [9]:
allel.opt.stats.gn_corrcoef_int8(gnb[0], gnb[1], gnb_sq[0], gnb_sq[1])

-0.002613014541566372

In [10]:
%timeit np.corrcoef(gnb)

The slowest run took 5.48 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 120 µs per loop


In [11]:
%timeit allel.opt.stats.gn_corrcoef_int8(gnb[0], gnb[1], gnb_sq[0], gnb_sq[1])

10000 loops, best of 3: 39.2 µs per loop


In [12]:
gnb2 = np.random.randint(0, 3, size=(1000, 1000)).astype('i1')

In [13]:
r2a = np.corrcoef(gnb2)
r2a

array([[ 1.        , -0.05577752,  0.00767298, ..., -0.03381146,
         0.02954236, -0.00755626],
       [-0.05577752,  1.        , -0.02363069, ...,  0.02232033,
         0.06444761,  0.03827147],
       [ 0.00767298, -0.02363069,  1.        , ..., -0.02565621,
         0.00159606,  0.01190215],
       ..., 
       [-0.03381146,  0.02232033, -0.02565621, ...,  1.        ,
        -0.01959978, -0.00503218],
       [ 0.02954236,  0.06444761,  0.00159606, ..., -0.01959978,
         1.        , -0.0092057 ],
       [-0.00755626,  0.03827147,  0.01190215, ..., -0.00503218,
        -0.0092057 ,  1.        ]])

In [14]:
r2b = allel.stats.rogers_huff_r(gnb2)
from scipy.spatial.distance import squareform
r2b_sq = squareform(r2b)
r2b_sq

array([[ 0.        , -0.05577756,  0.00767297, ..., -0.03381132,
         0.02954236, -0.00755618],
       [-0.05577756,  0.        , -0.02363073, ...,  0.02232023,
         0.06444771,  0.03827157],
       [ 0.00767297, -0.02363073,  0.        , ..., -0.02565622,
         0.00159614,  0.01190209],
       ..., 
       [-0.03381132,  0.02232023, -0.02565622, ...,  0.        ,
        -0.01959965, -0.00503222],
       [ 0.02954236,  0.06444771,  0.00159614, ..., -0.01959965,
         0.        , -0.00920562],
       [-0.00755618,  0.03827157,  0.01190209, ..., -0.00503222,
        -0.00920562,  0.        ]])

In [15]:
%timeit np.corrcoef(gnb2)
%memit np.corrcoef(gnb2)

10 loops, best of 3: 111 ms per loop
peak memory: 122.50 MiB, increment: 14.95 MiB


In [16]:
%timeit allel.stats.rogers_huff_r(gnb2)
%memit allel.stats.rogers_huff_r(gnb2)

1 loops, best of 3: 1.83 s per loop
peak memory: 107.73 MiB, increment: 0.05 MiB


In [17]:
import cProfile

In [18]:
cProfile.run('allel.stats.rogers_huff_r(gnb2)', sort='time')

         13 function calls in 1.843 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.842    1.842    1.842    1.842 {allel.opt.stats.gn_pairwise_corrcoef_int8}
        2    0.000    0.000    0.000    0.000 numeric.py:394(asarray)
        1    0.000    0.000    1.843    1.843 {built-in method exec}
        2    0.000    0.000    0.000    0.000 {built-in method array}
        1    0.000    0.000    1.842    1.842 ld.py:8(rogers_huff_r)
        1    0.000    0.000    0.000    0.000 util.py:20(asarray_ndim)
        1    0.000    0.000    1.842    1.842 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method hasattr}
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:2264(_handle_fromlist)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {method 'pop' of 'dict' objects}


